# Robust Identification of Investor Beliefs

by [Xiaohong Chen](https://economics.yale.edu/people/faculty/xiaohong-chen), [Lars Peter Hansen](http://larspeterhansen.org/) and [Peter G. Hansen](https://mitsloan.mit.edu/phd/students/peter-hansen).

The latest version of the paper can be found [here](http://larspeterhansen.org/research/papers/).

Notebook by: Han Xu, Zhenhuan Xie.

## 1. Overview

This notebook provides the source code and explanations for how we solve the dynamic problem in Section 5 of the paper. It also provides the source code for the figures in Section 6 as well as additional results that we did not report in the paper. Before we describe and implement the computation, let's first install and load necessary `Python packages` (or set up the server environment if you are running this notebook on `Goolge Colab`) by running the following cell.

In [1]:
# Check if the notebook is open in Google Colab
try:
    from google.colab import drive
    IN_COLAB = True
except:
    IN_COLAB = False

# Set up Google Colab environment
if IN_COLAB:
    import os
    # Link your Goolge Drive to Goolge Colab
    drive.mount('/content/gdrive')
    %cd '/content/gdrive/My Drive'
    # Create a folder to store our project
    if 'Belief_project' in os.listdir():
        %cd '/content/gdrive/My Drive/Belief_project'
    else:
        ! mkdir '/content/gdrive/My Drive/Belief_project/'
        %cd '/content/gdrive/My Drive/Belief_project/'
    # Clone GitHub repo to the folder and change working directory to the repo
    if 'Beliefs' not in os.listdir():
        ! git clone https://github.com/lphansen/Beliefs.git
    %cd '/content/gdrive/My Drive/Belief_project/Beliefs'
    
# Set up local environment
else:
    try:
        import plotly
    except:
        import sys
        !{sys.executable} -m pip install plotly

# Load packages
import time
from source.utilities import *
from source.plotting_module import *
print('----------Successfully Loaded Python Packages----------')

# Test
import time
from source.preprocessing import preprocess_data
from source.solver import solve, find_ξ

----------Successfully Loaded Python Packages----------


In [2]:
f, log_Rw, z0, z1 = preprocess_data()

result_min = solve(f=f, g=log_Rw, z0=z0, z1=z1, 
                   ξ=100., n_states=3, tol=1e-9, max_iter=1000)

ξ = find_ξ(solver_args=(f, log_Rw, z0, z1, 3, 1e-9, 1000),
           min_RE=result_min['RE'], pct=0.2, initial_guess=1.,
           interval=(0, 100.), tol=1e-5, max_iter=100)

1.0
1 100.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0
0.19561728207430007
1 1.0


KeyboardInterrupt: 

In [ ]:
ξ

## 2. Moment Bounds

Recall **Problem 5.5** in the paper.  For a real number $\mu$ and a random variable $v_0$, 

\begin{equation}
\mu = \min_{N_1 \ge 0} \mathbb{E}\left(N_1\left[g(X_1)+\xi\log N_1 + v_1\right]\mid \mathfrak{I}_0\right) - v_0
\end{equation}
*subject to constraints*:
\begin{align*}
\mathbb{E}\left[N_1 f(X_1)\mid\mathfrak{I}_0\right] &= 0\\
\mathbb{E}\left[N_1 \mid \mathfrak{I}_0\right] &= 1
\end{align*}
where $v_1$ is a version of $v_0$ shifted forward one time period.

By **Proposition 5.7**, this problem can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_{\lambda_0}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\lambda_0\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{I}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

The optimized results will depend on the choice of $\xi$. Alternative values of $\xi$ imply alternative bounds on the expectation of $g(X_1)$ and the corresponding relative entropy.  Below is an illustration of how the minimized objectives $\mu^*$ and $\epsilon^*$ change with $\xi$. Data and calculation details are described later in Section 3. 

In [ ]:
objective_vs_ξ(n_states=3)

The implied solution for the probablity distortion is:

\begin{equation}
N_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\lambda^*_0(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

where $\lambda^*_0$ is the optimizing choice for $\lambda_0$ and $\left(\epsilon^*,e_0^*\right)$ are selected so that the resulting $\sf Q$ induces stochastically stable. The conditional expectation implied by the bound is

\begin{equation}
\mathbb{E}\left[N_1^*g(X_1)\mid \mathfrak{I}_0\right]
\end{equation}

which in turn implies a bound on the unconditional expectation equal to

\begin{equation}
\int \mathbb{E}\left[N_1^*g(X_1)\mid\mathfrak{I}_0\right]d \sf Q_0^*
\end{equation}

The implied relative entropy is

\begin{equation}
\int \mathbb{E}\left(N_1^*\log N_1^*\mid \mathfrak{I}_0\right)d \sf Q_0^*
\end{equation}

To compute the bounds on the expected logarithmic return on market, we let the logarithm of this return on wealth be our $g$; 

To compute the bounds on risk premium and generalized volatility, we extend the previous approach as follows:

### 2.1 Bounding Risk Premia

- Set $g(X_1)=R^w_1-\zeta R^f_1$ where $\zeta$ is a "multiplier" that we will search over;


- for alternative $\zeta$, deduce $N_1^*(\zeta)$ and $\sf Q_0^*(\zeta)$ as described in the paper;


- compute:

$$
\log \int \mathbb{E}\left[N_1^*(\zeta)R^w_1\mid \mathfrak{I}_0\right]d \sf Q_0^*(\zeta) - \log \int \mathbb{E}\left[N_1^*(\zeta)R^f_1\mid \mathfrak{I}_0\right]d \sf Q_0^*(\zeta)
$$
and minimize with respect to $\zeta$;


- set $g(X_1)=-R^w_1+\zeta R^f_1$, repeat, and use the negative of the minimizer to obtain the upper bound.

### 2.2 Bounding Volatility

We show how to bound an entropic measure of volatility.  Other measures could be computed using a similar approch.  

- Set $g(X_1)=R^w_1-\zeta \log R^w_1$ where $\zeta$ is a "multiplier" that we will search over;


- for alternative $\zeta$, deduce $N_1^*(\zeta)$ and $\sf Q_0^*(\zeta)$ as described in the paper;


- compute:

$$
\log \int \mathbb{E}\left[N_1^*(\zeta)R^w_1\mid \mathfrak{I}_0\right]d {\sf Q_0}^*(\zeta) - \int \mathbb{E}\left[N_1^*(\zeta)\log R^w_1\mid \mathfrak{I}_0\right]d {\sf Q_0}^*(\zeta)
$$
and minimize with respect to $\zeta$;


- set $g(X_1)=-R^w_1+\zeta \log R^w_1$, repeat, and use the negative of the minimizer to obtain the upper bound.

## 3. Code Implementation

### 3.1 Data
The file “UnitaryData.csv” contains the following data from 1954-2016:

- The first four columns contain Euler equation errors from the unitary risk aversion model corresponding to the 3-month T-bill rate, the market excess return, the SMB excess return, and the HML excess return respectively. Under a feasible belief distortion, all four of these variables should have expectation of zero (conditional or unconditional).


- The column “d.p” contains the dividend-price ratio for the CRSP value-weighted index, computed at the start of the return period. Hence functions of d.p[i] (i.e. quantile indicator functions) are valid instruments for the returns in row i.


- The final column “log.RW” contains values of the logarithmic return on CRSP value-weighted index. We use this as a proxy for the logarithmic return on wealth. This is the random variable whose expectation we are intersted in bounding.

All returns are quarterly and inflation-adjusted.

In [ ]:
# Load data
data = pd.read_csv('data/UnitaryData.csv')
# Show statistics of the data
data.describe()

Given our direct use of dividend-price measures, we purposefully choose a coarse conditioning of information and split the dividend price ratios into $n$ bins using the $n$ empirical terciles. We take the dividend-price terciles to be a $n$-state Markov process. Then we multiply each of the first four columns by each of the $n$ columns of the indicator function of dividend-price terciles to form a $4n$-dimensional $f$. 

When bounding the expected logarithmic return on wealth, we take $\log R^w$ as our $g$. When bounding the risk premium and volatility, we define $g$ as discussed above in Section 2.1 and 2.2. 

### 3.2 Computational Strategy

Since we have $n$ distinct states in our application, we can represent the function $e(\cdot)$ as a $n$-dimensional vector. Additionally, we are free to impose the normalization $e_1=1$. We can solve the dual problem numerically by something analogous to value function iteration for $e=(1,e_2,...,e_n)$. Here is the iteration scheme:


1\. Guess $e={\mathbb{1}}_{n\times 1}$.

2\. For $k \in \{1,2,...,n\}$, solve
\begin{equation}
v_k = \min_{\lambda_0} \hat{\mathbb{E}}\left(\exp \left[-\frac{1}{\xi}g(X_1) + \lambda_0f(X_1)\right]e(Z_0)\mid Z_0 = k\right)
\end{equation}

3\. Store
\begin{align*}
\hat{e} &= v/v_1 \\
\hat{\epsilon} &= v_1 \\
\text{error} &= \|\hat{e}-e\|
\end{align*}

4\. Set $e = \hat{e}$.

5\. Iterate steps 2-4 until error is smaller than $10^{-9}$.

Once we have (approximately) stationary values for $\epsilon^*$ and $e^*$ as well as the optimizing $\lambda_0^*$, we can form the conditional belief distortion
\begin{equation}
N_1 = \frac{1}{\epsilon^*} \exp \left[-\frac{1}{\xi}g(X_1)+\lambda_0^* \cdot f(X_1)\right]\frac{e^*(Z_1)}{e^*(Z_0)}
\end{equation}

To obtain the unconditional relative entropy, we need to average across states using the implied stationary distribution coming from the distorted probabilities. Define a $n\times n$ matrix $\tilde{P}$ by 
$$
\tilde{P}_{i,j} = \hat{\mathbb{E}}\left[N_1 \mathcal{1}\left(Z_1 = j\right)\mid Z_0 = i\right]
$$

We should have that $\tilde{P}$ is a transition probability matrix, so $\tilde{P}\mathbb{1}=\mathbb{1}$. Next, solve for the stationary distribution $\pi\in \mathbb{R}^n$ as the dominant left eigenvector of $\tilde{P}$, i.e.
\begin{equation}
\tilde{\pi}^\prime \tilde{P} = \tilde{\pi}^\prime
\end{equation}

Then, the unconditional relative entropy can be computed as
\begin{equation}
\text{RE}(\xi) = \sum_{k=1}^{n}\hat{\mathbb{E}}\left[N_1\log N_1 \mid Z_0 = k\right]\cdot \tilde{\pi}_k
\end{equation}


Note: in the following code implementation, we set $n=3$ as used in the paper. Users can specify a different $n$ by changing the `n_states` argument.

In [ ]:
f, log_Rw, z0, z1 = preprocess_data()



# # Count time
# time_start = time.time() 

# # Initialize the solver
# n_states = 3 # User can specify a different number of states here
# solver = InterDivConstraint(n_states = n_states,tol=1e-9,max_iter=1000)

# # Define g(X) = log Rw
# solver.g = solver.log_Rw

# # Find ξ that corresponds to 120% min RE
# x_min_RE = 1.2
# ξ_lower = solver.find_ξ(x_min_RE=x_min_RE,lower=True,tol=1e-7,max_iter=100)
# ξ_upper = solver.find_ξ(x_min_RE=x_min_RE,lower=False,tol=1e-7,max_iter=100)

# # Solve models with the chosen ξ
# result_lower = solver.iterate(ξ_lower,lower=True)
# result_upper = solver.iterate(ξ_upper,lower=False)

# # Print out the time spent
# time_spent = round(time.time()-time_start,4)

### 3.3 Results

In [ ]:
# Print iteration information
print("--- Iteration Ends ---")
print("%sx min rel entropy" % x_min_RE)
print("Iterations (lower bound problem): %s" % (result_lower['count']))
print("Time spent: %s seconds" % (np.round(time_spent,2)))

# Print converged parameter results
print("\n")
print("--- Converged values for the lower bound problem ---")
print("ϵ: %s" % np.round(result_lower['ϵ'],2))
print("e: %s" % np.round(result_lower['e'],2))
print("λ: %s" % np.round(result_lower['λ'],2))

print(" ")
print("--- Converged values for the upper bound problem ---")
print("ϵ: %s" % np.round(result_upper['ϵ'],2))
print("e: %s" % np.round(result_upper['e'],2))
print("λ: %s" % np.round(result_upper['λ'],2))

# Print transition probability matrix under the original empirical probability
print("\n")
print("--- Transition Probability Matrix (Original) ---")
print(np.round(result_lower['P'],2))

# Print transition probability matrix under distorted probability, lower bound
print(" ")
print("--- Transition Probability Matrix (Distorted, lower bound problem) ---")
print(np.round(result_lower['P_tilde'],2))

# Print transition probability matrix under distorted probability, upper bound
print(" ")
print("--- Transition Probability Matrix (Distorted, upper bound problem) ---")
print(np.round(result_upper['P_tilde'],2))


# Print stationary distribution under the original empirical probability
print("\n")
print("--- Stationary Distribution (Original) ---")
print(np.round(result_lower['π'],2))

# Print stationary distribution under distorted probability, lower bound
print(" ")
print("--- Stationary Distribution (Distorted, lower bound problem) ---")
print(np.round(result_lower['π_tilde'],2))

# Print stationary distribution under distorted probability, upper bound
print(" ")
print("--- Stationary Distribution (Distorted, upper bound problem) ---")
print(np.round(result_upper['π_tilde'],2))

# Print relative entropy
print("\n")
print("--- Relative Entropy (lower bound problem) ---")
for state in np.arange(1, n_states+1):
    print(f"E[NlogN|state {state}] = {np.round(result_lower['RE_cond'][state-1],4)}")
print("E[NlogN]         = %s " % np.round(result_lower['RE'],4))

# Print relative entropy
print(" ")
print("--- Relative Entropy (Upper bound problem) ---")
for state in np.arange(1, n_states+1):
    print(f"E[NlogN|state {state}] = {np.round(result_upper['RE_cond'][state-1],4)}")
print("E[NlogN]         = %s " % np.round(result_upper['RE'],4))

# Print conditional moment & bounds
print("\n")
print("--- Moment (Original, annualized, %) ---")
for state in np.arange(1, n_states+1):
    print(f"E[g(X)|state {state}] = {np.round(result_lower['moment_cond'][state-1]*400,2)}")
print("E[g(X)]  = %s " % (np.round(result_lower['moment']*400,2)))
print(" ")
print("--- Moment (Lower bound, annualized, %) ---")
for state in np.arange(1, n_states+1):
    print(f"E[Ng(X)|state {state}] = {np.round(result_lower['moment_bound_cond'][state-1]*400,2)}")
print("E[Ng(X)] = %s " % (np.round(result_lower['moment_bound']*400,2)))
print(" ")
print("--- Moment (Upper bound, annualized, %) ---")
for state in np.arange(1, n_states+1):
    print(f"E[Ng(X)|state {state}] = {np.round(result_upper['moment_bound_cond'][state-1]*400,2)}")
print("E[Ng(X)] = %s " % (np.round(result_upper['moment_bound']*400,2)))

In [ ]:
time_start = time.time() 
entropy_moment_bounds(n_states)
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

## 4. Tables and plots

In [ ]:
# Table 1: transition matrix and stationary probability
n_states = 3 # user can specify a different number of states here
solver = InterDivConstraint(n_states=n_states,tol=1e-9,max_iter=1000)
solver.g = solver.log_Rw
result_min = solver.iterate(ξ=100.,lower=True)

print('Table 1: Empirical and distorted transition probabilities')
print('-------------------------------------------------------')
print('               empirical             min entropy')
print('-------------------------------------------------------')
print('Transition Matrix:')
for state in np.arange(1, n_states+1):
    print(f"          {np.round(result_min['P'][state-1],2)}       {np.round(result_min['P_tilde'][state-1],2)}")
print('')
print('Stationary Probability:')
print(f"          {np.round(result_min['π'],2)}       {np.round(result_min['π_tilde'],2)}")
print('-------------------------------------------------------')

In [ ]:
# Figure 1: bounds on expected log market return
# Below we load presolved ξs but users can use solver.find_ξ to resolve them.
# Example: solver.find_ξ(x_min_RE=1.2,lower=True,tol=1e-7,max_iter=100)

time_start = time.time()
solver = InterDivConstraint(n_states=3,tol=1e-9,max_iter=1000)
solver.g = solver.log_Rw

result_lower_list = []
result_upper_list = []

for i in range(0,7):
    result_lower_list.append(solver.iterate(ξs_lower[i],lower=True))
    result_upper_list.append(solver.iterate(ξs_upper[i],lower=False))

def f1(percent):
    box_chart(result_min,result_lower_list[int(percent/5)],result_upper_list[int(percent/5)])

print('Figure 1: Expected log market return')
interact(f1, percent=widgets.IntSlider(min=0, max=30, step=5, value=20));
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

In [ ]:
# Figure 2: bounds on risk premia
n_states = 3
# Case 1: min RE
risk_premia_min, risk_premia_cond_min, risk_premia_empirical, risk_premia_cond_empirical = risk_premia(n_states,1.,x_min_RE=1.,lower=True,ξ_tol=1e-7)
result_risk_min = {'moment':risk_premia_empirical,'moment_cond':risk_premia_cond_empirical,'moment_bound':risk_premia_min,'moment_bound_cond':risk_premia_cond_min}

# Below we load presolved ζs but users can plot the objective function over ζ to refind the optimal ζs. 

# Case 2: 20% higher RE
time_start = time.time()
result_risk_lower_list = []
result_risk_upper_list = []
for i in range(0,7):
    # lower bound
    risk_premia_lower, risk_premia_cond_lower, _ ,_ = risk_premia(n_states,ζs_lower[i],x_min_RE=1.+i*0.05,lower=True,ξ_tol=1e-7)
    result_risk_lower = {'moment_bound':risk_premia_lower,'moment_bound_cond':risk_premia_cond_lower}
    result_risk_lower_list.append(result_risk_lower)
    # upper bound
    risk_premia_upper, risk_premia_cond_upper, _ ,_ = risk_premia(n_states,ζs_upper[i],x_min_RE=1.+i*0.05,lower=False,ξ_tol=1e-7)
    result_risk_upper = {'moment_bound':risk_premia_upper,'moment_bound_cond':risk_premia_cond_upper}
    result_risk_upper_list.append(result_risk_upper)
    
def f2(percent):
    box_chart(result_risk_min,result_risk_lower_list[int(percent/5)],result_risk_upper_list[int(percent/5)])
    
print("Figure 2: Proportional risk compensations")
interact(f2, percent=widgets.IntSlider(min=0, max=30, step=5, value=20));
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

In [ ]:
# Table 2: bounds on log expected return and generalized volatility

# Calculate bounds on expected return
solver = InterDivConstraint(n_states=3,tol=1e-9,max_iter=1000)
solver.g = np.exp(solver.log_Rw)
x_min_RE = 1.2
ξ_lower = solver.find_ξ(x_min_RE=x_min_RE,lower=True,tol=1e-7,max_iter=100)
ξ_upper = solver.find_ξ(x_min_RE=x_min_RE,lower=False,tol=1e-7,max_iter=100)
result_lower = solver.iterate(ξ_lower,lower=True)
result_upper = solver.iterate(ξ_upper,lower=False)
result_min   = solver.iterate(ξ=100.)

# Calculate bounds on generalized volatility
vol_min, vol_cond_min, vol_empirical, vol_cond_empirical = volatility(3,1.,x_min_RE=1.,lower=True,ξ_tol=1e-7)
vol_lower, vol_cond_lower, _, _ = volatility(3,1.008,x_min_RE,lower=True,ξ_tol=1e-7)
vol_upper, vol_cond_upper, _, _ = volatility(3,1.009,x_min_RE,lower=False,ξ_tol=1e-7)

print('Table 2: Log expected market return and generalized volatility')
print('-------------------------------------------------------------------------------')
print('conditioning      logE           logE          logE - Elog     logE - Elog')
print('                empirical       imputed         empirical        imputed')
print('                             (lower, upper)                   (lower,upper)')
print('-------------------------------------------------------------------------------')
print('low D/P           %s           %s             %s            %s' \
      % (np.round(np.log(result_min['moment_cond'][0])*400,2),np.round(np.log(result_min['moment_bound_cond'][0])*400,2),np.round(vol_cond_empirical[0]*400,2),np.round(vol_cond_min[0]*400,2)))
print('                              (%s,%s)                      (%s,%s)' \
      % (np.round(np.log(result_lower['moment_bound_cond'][0])*400,2),np.round(np.log(result_upper['moment_bound_cond'][0])*400,2),np.round(vol_cond_lower[0]*400,2),np.round(vol_cond_upper[0]*400,2)))
print('mid D/P           %s            %s             %s            %s' \
      % (np.round(np.log(result_min['moment_cond'][1])*400,2),np.round(np.log(result_min['moment_bound_cond'][1])*400,2),np.round(vol_cond_empirical[1]*400,2),np.round(vol_cond_min[1]*400,2)))
print('                              (%s,%s)                      (%s,%s)' \
      % (np.round(np.log(result_lower['moment_bound_cond'][1])*400,2),np.round(np.log(result_upper['moment_bound_cond'][1])*400,2),np.round(vol_cond_lower[1]*400,2),np.round(vol_cond_upper[1]*400,2)))
print('high D/P          %s          %s             %s            %s' \
      % (np.round(np.log(result_min['moment_cond'][2])*400,2),np.round(np.log(result_min['moment_bound_cond'][2])*400,2),np.round(vol_cond_empirical[2]*400,2),np.round(vol_cond_min[2]*400,2)))
print('                              (%s,%s)                      (%s,%s)' \
      % (np.round(np.log(result_lower['moment_bound_cond'][2])*400,2),np.round(np.log(result_upper['moment_bound_cond'][2])*400,2),np.round(vol_cond_lower[2]*400,2),np.round(vol_cond_upper[2]*400,2)))
print('none              %s           %s             %s            %s' \
      % (np.round(np.log(result_min['moment'])*400,2),np.round(np.log(result_min['moment_bound'])*400,2),np.round(vol_empirical*400,2),np.round(vol_min*400,2)))
print('                              (%s,%s)                      (%s,%s)' \
      % (np.round(np.log(result_lower['moment_bound'])*400,2),np.round(np.log(result_upper['moment_bound'])*400,2),np.round(vol_lower*400,2),np.round(vol_upper*400,2)))
print('-------------------------------------------------------------------------------')
print('note: the numbers in the parentheses impose a relative entropy constraint')
print('      that is %s percent higher than the minimum.' % (int(x_min_RE*100-100)))